<h1 align="center">1. Базовые понятия статистики</h1>

<h3 align="center">Задание 1. Загрузка данных</h3>

Изучить представленный набор данных на основе описания его столбцов, загрузить его и оставить 8 столбцов для дальнейшего изучения: surgery?, Age, rectal temperature, pulse, respiratory rate, temperature of extremities, pain, outcome.

In [202]:
import pandas as pd
import numpy as np

In [203]:
cols = ['surgery?', 'Age', 'rectal_temperature', 'pulse', 'respiratory_rate','temperature_of_extremities', 'pain', 'outcome']

df = pd.read_csv('https://raw.githubusercontent.com/obulygin/pyda_homeworks/master/statistics_basics/horse_data.csv',
                 header=None, na_values='?', usecols = [0, 1, 3, 4, 5, 6, 10, 22], names = cols)
df.info()
df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 8 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   surgery?                    299 non-null    float64
 1   Age                         300 non-null    int64  
 2   rectal_temperature          240 non-null    float64
 3   pulse                       276 non-null    float64
 4   respiratory_rate            242 non-null    float64
 5   temperature_of_extremities  244 non-null    float64
 6   pain                        245 non-null    float64
 7   outcome                     299 non-null    float64
dtypes: float64(7), int64(1)
memory usage: 18.9 KB


,surgery?,Age,rectal_temperature,pulse,respiratory_rate,temperature_of_extremities,pain,outcome
0,2.0,1,38.5,66.0,28.0,3.0,5.0,2.0
1,1.0,1,39.2,88.0,20.0,NaN,3.0,3.0
2,2.0,1,38.3,40.0,24.0,1.0,3.0,1.0
3,1.0,9,39.1,164.0,84.0,4.0,2.0,2.0
4,2.0,1,37.3,104.0,35.0,NaN,NaN,2.0
...,...,...,...,...,...,...,...,...
295,1.0,1,NaN,120.0,70.0,4.0,2.0,3.0
296,2.0,1,37.2,72.0,24.0,3.0,4.0,3.0
297,1.0,1,37.5,72.0,30.0,4.0,4.0,2.0
298,1.0,1,36.5,100.0,24.0,3.0,3.0,1.0


In [204]:
df.describe()

,surgery?,Age,rectal_temperature,pulse,respiratory_rate,temperature_of_extremities,pain,outcome
count,299.000000,300.000000,240.000000,276.000000,242.000000,244.000000,245.00000,299.000000
mean,1.397993,1.640000,38.167917,71.913043,30.417355,2.348361,2.95102,1.551839
std,0.490305,2.173972,0.732289,28.630557,17.642231,1.045054,1.30794,0.737187
min,1.000000,1.000000,35.400000,30.000000,8.000000,1.000000,1.00000,1.000000
25%,1.000000,1.000000,37.800000,48.000000,18.500000,1.000000,2.00000,1.000000
50%,1.000000,1.000000,38.200000,64.000000,24.500000,3.000000,3.00000,1.000000
75%,2.000000,1.000000,38.500000,88.000000,36.000000,3.000000,4.00000,2.000000
max,2.000000,9.000000,40.800000,184.000000,96.000000,4.000000,5.00000,3.000000


<h3 align="center">Задание 2. Первичное изучение данных</h3>

Проанализировать значения по столбцам, рассчитать базовые статистики, найти выбросы.

In [205]:
df_max = df.select_dtypes(include='number').max()
df_min = df.select_dtypes(include='number').min()
df_range = df_max - df_min
df_mean = df.select_dtypes(include='number').mean()
df_mode = df.select_dtypes(include='number').mode()
df_median = df.select_dtypes(include='number').median()
df_std = df.select_dtypes(include='number').std()
df_var = df.select_dtypes(include='number').var().round(2)

df_quantile_1_3 = df.select_dtypes(include='number').quantile([0.25, 0.75])
df_quantile_random = df.select_dtypes(include='number').quantile(0.33)
df_quantile_1 = df.select_dtypes(include='number').quantile(0.25)
df_quantile_3 = df.select_dtypes(include='number').quantile(0.75)
iqr = df_quantile_3 - df_quantile_1

lower_bound = df_quantile_1 - (1.5 * iqr) 
upper_bound = df_quantile_3 + (1.5 * iqr)

print('МАКСИМУМ', '\n', df_max, '\n', '\n', 'МИНИМУМ', '\n', df_min,  '\n', '\n', 'РАЗМАХ', '\n', df_range,  
      '\n', '\n', 'СРЕДНЕЕ АРИФМЕТИЧЕСКОЕ', '\n', df_mean, '\n', '\n','МЕДИАНА', '\n', df_median,  '\n', '\n', 
      'СТАНДАРТНОЕ ОТКЛОНЕНИЕ', '\n', df_std, '\n', '\n', 'ДИСПЕРСИЯ', '\n', df_var, '\n',  '\n', 'МОДА', '\n', df_mode, 
      '\n',  '\n', 'ПЕРВЫЙ КВАРТИЛЬ', '\n', df_quantile_1, '\n',  '\n', 'ТРЕТИЙ КВАРТИЛЬ', '\n', df_quantile_3, 
      '\n',  '\n', 'ПРОИЗВОЛЬНЫЙ ПЕРЦЕНТИЛЬ', '\n', df_quantile_random, '\n',  '\n', 'МЕЖКВАРТИЛЬНЫЙ РАЗМАХ', '\n', iqr)

МАКСИМУМ 
 surgery?                        2.0
Age                             9.0
rectal_temperature             40.8
pulse                         184.0
respiratory_rate               96.0
temperature_of_extremities      4.0
pain                            5.0
outcome                         3.0
dtype: float64 
 
 МИНИМУМ 
 surgery?                       1.0
Age                            1.0
rectal_temperature            35.4
pulse                         30.0
respiratory_rate               8.0
temperature_of_extremities     1.0
pain                           1.0
outcome                        1.0
dtype: float64 
 
 РАЗМАХ 
 surgery?                        1.0
Age                             8.0
rectal_temperature              5.4
pulse                         154.0
respiratory_rate               88.0
temperature_of_extremities      3.0
pain                            4.0
outcome                         2.0
dtype: float64 
 
 СРЕДНЕЕ АРИФМЕТИЧЕСКОЕ 
 surgery?                       1

<h3 align="center">Задание 3. Работа с пропусками</h3>

Рассчитать количество пропусков для всех выбранных столбцов. Принять и обосновать решение о методе заполнения пропусков по каждому столбцу на основе рассчитанных статистик и возможной взаимосвязи значений в них. Сформировать датафрейм, в котором пропуски будут отсутствовать.

In [206]:
df.isna().sum()

surgery?                       1
Age                            0
rectal_temperature            60
pulse                         24
respiratory_rate              58
temperature_of_extremities    56
pain                          55
outcome                        1
dtype: int64

In [207]:
df.isnull().head(n=60)

,surgery?,Age,rectal_temperature,pulse,respiratory_rate,temperature_of_extremities,pain,outcome
0,False,False,False,False,False,False,False,False
1,False,False,False,False,False,True,False,False
2,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False
4,False,False,False,False,False,True,True,False
5,False,False,True,True,True,False,False,False
6,False,False,False,False,False,False,False,False
7,False,False,True,False,True,False,True,False
8,False,False,True,False,False,False,False,False
9,False,False,False,False,True,False,False,False


### surgery? outcome - заменим None на mode, так как количество пустых ячеек незначительно, по одной

In [208]:
df['surgery?'] = df['surgery?'].fillna(df['surgery?'].mode()[0])
df['outcome'] = df['outcome'].fillna(df['outcome'].mode()[0])

### pain  и  outcome - исходя из сути датафрейма, свяжем эти два столбца и заменим None на median

In [209]:
df['pain'] = df['pain'].fillna(df.groupby('outcome')['pain'].transform('median'))

### temperature_of_extremities и pain - очевидно связаны, заменим None на median

In [210]:
df['temperature_of_extremities'] = df['temperature_of_extremities'].fillna(df.groupby('pain')['temperature_of_extremities'].transform('median'))

### temperature_of_extremities и pulse - имеют между собой зависимость, заменим None на median

In [211]:
df['pulse'] = df['pulse'].fillna(df.groupby('pain')['pulse'].transform('median'))

### rectal temperature и pain - свяжем эти два значения, заменим None на median

In [212]:
df['rectal_temperature'] = df['rectal_temperature'].fillna(df.groupby('pain')['rectal_temperature'].transform('median'))

### respiratory_rate - удалим, так как зависимость от этого параметра нам неизвестна

In [213]:
df_finall = df.drop(['respiratory_rate'], axis=1)
df_finall.head(60)

,surgery?,Age,rectal_temperature,pulse,temperature_of_extremities,pain,outcome
0,2.0,1,38.5,66.0,3.0,5.0,2.0
1,1.0,1,39.2,88.0,2.0,3.0,3.0
2,2.0,1,38.3,40.0,1.0,3.0,1.0
3,1.0,9,39.1,164.0,4.0,2.0,2.0
4,2.0,1,37.3,104.0,3.0,4.0,2.0
5,2.0,1,38.3,78.0,2.0,2.0,1.0
6,1.0,1,37.9,48.0,1.0,3.0,1.0
7,1.0,1,38.0,60.0,3.0,4.0,2.0
8,2.0,1,38.0,80.0,3.0,4.0,3.0
9,2.0,9,38.3,90.0,1.0,5.0,1.0


In [214]:
df_finall.isna().sum()

surgery?                      0
Age                           0
rectal_temperature            0
pulse                         0
temperature_of_extremities    0
pain                          0
outcome                       0
dtype: int64

![](Pandas_logo.png)